In [10]:
from github import Github
from collections import Counter, defaultdict
from pydriller import Repository
import os
import git  
import shutil
import matplotlib.pyplot as plt
import os

## **Achando estatistícas das top 5 linguagens presentes no perfil do dev, seus respectivos repositórios e numero de commits relacionados a cada um**

In [ ]:
GITHUB_TOKEN = "my token"
USERNAME = "johnatan-si"

g = Github(GITHUB_TOKEN)
user = g.get_user(USERNAME)

print(f"\n📛 Nome: {user.name}")
print(f"🔐 Login: {user.login}")
print(f"🖼️ Foto de perfil: {user.avatar_url}\n")

fotoPerfil = user.avatar_url
languageCommitCounter = Counter()
reposLanguage = defaultdict(list)

for repo in user.get_repos():
    try:
        commits = repo.get_commits()
        commit_count = commits.totalCount
        langs = repo.get_languages()
        repo_name = repo.name
        repo_url = repo.clone_url

        for lang in langs:
            languageCommitCounter[lang] += commit_count
            reposLanguage[lang].append({
                'name': repo_name,
                'commits': commit_count,
                'locAdd': 0,        
                'locRemov': 0,
                'url': repo_url     
            })

    except Exception as e:
        print(f"⚠️ Erro ao acessar {repo.name}: {e}")

top_5_languages = languageCommitCounter.most_common(5)
top5ReposLanguage = {lang: reposLanguage[lang] for lang, _ in top_5_languages}

print("\n📊 Top 5 linguagens mais ativas (por número de commits):")
for lang, total_commits in top_5_languages:
    print(f"➡️ {lang}: {total_commits} commits")

print("\n📂 Repositórios por linguagem (Top 5):")
for lang, repos in top5ReposLanguage.items():
    print(f"\n🗂️ {lang}:")
    for repo in repos:
        print(f"   - {repo['name']}: {repo['commits']} commits, +{repo['locAdd']} / -{repo['locRemov']} LOC")



📛 Nome: Johnatan 
🔐 Login: johnatan-si
🖼️ Foto de perfil: https://avatars.githubusercontent.com/u/8726738?v=4

⚠️ Erro ao acessar SourceLearning-: Git Repository is empty.: 409 {"message": "Git Repository is empty.", "documentation_url": "https://docs.github.com/rest/commits/commits#list-commits", "status": "409"}

📊 Top 5 linguagens mais ativas (por número de commits):
➡️ Java: 246 commits
➡️ Dockerfile: 111 commits
➡️ Python: 81 commits
➡️ Makefile: 59 commits
➡️ AspectJ: 58 commits

📂 Repositórios por linguagem (Top 5):

🗂️ Java:
   - astar0: 2 commits, +0 / -0 LOC
   - AulasPUCMG: 60 commits, +0 / -0 LOC
   - demo-ci: 56 commits, +0 / -0 LOC
   - DownloadGit: 2 commits, +0 / -0 LOC
   - extractgit: 1 commits, +0 / -0 LOC
   - GitBlame: 8 commits, +0 / -0 LOC
   - JExpert: 1 commits, +0 / -0 LOC
   - JExpert2.0: 1 commits, +0 / -0 LOC
   - jreuse: 12 commits, +0 / -0 LOC
   - JSERD2020: 43 commits, +0 / -0 LOC
   - LineCounter: 2 commits, +0 / -0 LOC
   - ufmg.reuso.software: 58 co

In [12]:
output_dir = "graficosCommits"
os.makedirs(output_dir, exist_ok=True)

print("\n📊 Gerando gráficos por linguagem...")

for lang, repos in top5ReposLanguage.items():
    nomes_repos = []
    commits = []

    for repo in repos:
        nomes_repos.append(repo['name'])
        commits.append(repo['commits'])

    plt.figure(figsize=(10, 6))

    bar_width = 0.4
    x_pos = range(len(nomes_repos))

    bars = plt.bar(x_pos, commits, color='skyblue', width=bar_width)

    plt.xticks(x_pos, nomes_repos, rotation=45, ha='right')
    plt.xlim(-0.5, len(nomes_repos) - 0.5)

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2., height,
                 f'{int(height)}',
                 ha='center', va='bottom')

    plt.title(f"Commits por repositório - {lang}")
    plt.xlabel("Repositórios")
    plt.ylabel("Commits")
    plt.tight_layout()

    filepath = os.path.join(output_dir, f"{lang}_commits.png")
    plt.savefig(filepath)
    plt.close()

    print(f"✅ Gráfico salvo: {filepath}")



📊 Gerando gráficos por linguagem...
✅ Gráfico salvo: graficosCommits/Java_commits.png
✅ Gráfico salvo: graficosCommits/Dockerfile_commits.png
✅ Gráfico salvo: graficosCommits/Python_commits.png
✅ Gráfico salvo: graficosCommits/Makefile_commits.png
✅ Gráfico salvo: graficosCommits/AspectJ_commits.png


## **Clonando repositórios das top 5 linguagens e fazendo contagem de locAdd e locRemov de cada um e gerando graficos respectivos para cada linguagem.**

In [13]:
CLONE_DIR = "gitClones"

g = Github(GITHUB_TOKEN)
user = g.get_user(USERNAME)

print(f"\n📛 Nome: {user.name}")
print(f"🔐 Login: {user.login}")
print(f"🖼️ Foto de perfil: {user.avatar_url}\n")

fotoPerfil = user.avatar_url
languageCommitCounter = Counter()
reposLanguage = defaultdict(list)

for repo in user.get_repos():
    try:
        commits = repo.get_commits()
        commit_count = commits.totalCount
        langs = repo.get_languages()
        clone_url = repo.clone_url
        repo_name = repo.name
        repo_path = os.path.join(CLONE_DIR, repo_name)

        for lang in langs:
            languageCommitCounter[lang] += commit_count
            reposLanguage[lang].append({
                'name': repo_name,
                'commits': commit_count,
                'url': clone_url
            })

    except Exception as e:
        print(f"⚠️ Erro ao acessar {repo.name}: {e}")

top_5_languages = languageCommitCounter.most_common(5)
top5ReposLanguage = defaultdict(list)

print("\n🔁 Clonando repositórios e analisando com PyDriller...")

os.makedirs(CLONE_DIR, exist_ok=True)

for lang, _ in top_5_languages:
    for repo_info in reposLanguage[lang]:
        repo_name = repo_info['name']
        commit_count = repo_info['commits']
        url = repo_info['url']
        local_path = os.path.join(CLONE_DIR, repo_name)

        if not os.path.exists(local_path):
            try:
                print(f"🔽 Clonando {repo_name}...")
                git.Repo.clone_from(url, local_path)
            except Exception as e:
                print(f"❌ Falha ao clonar {repo_name}: {e}")
                continue

        loc_add = 0
        loc_remov = 0

        try:
            for commit in Repository(local_path).traverse_commits():
                for mod in commit.modified_files:
                    loc_add += mod.added_lines
                    loc_remov += mod.deleted_lines
        except Exception as e:
            print(f"❌ Erro analisando {repo_name}: {e}")
            continue

        top5ReposLanguage[lang].append({
            'name': repo_name,
            'commits': commit_count,
            'locAdd': loc_add,
            'locRemov': loc_remov
        })

print("\n📂 Repositórios por linguagem com commits e LOC:")
for lang, repos in top5ReposLanguage.items():
    print(f"\n🗂️ {lang}:")
    for repo in repos:
        print(f"   - {repo['name']}: {repo['commits']} commits, +{repo['locAdd']} / -{repo['locRemov']} LOC")



📛 Nome: Johnatan 
🔐 Login: johnatan-si
🖼️ Foto de perfil: https://avatars.githubusercontent.com/u/8726738?v=4

⚠️ Erro ao acessar SourceLearning-: Git Repository is empty.: 409 {"message": "Git Repository is empty.", "documentation_url": "https://docs.github.com/rest/commits/commits#list-commits", "status": "409"}

🔁 Clonando repositórios e analisando com PyDriller...

📂 Repositórios por linguagem com commits e LOC:

🗂️ Java:
   - astar0: 2 commits, +1649 / -31 LOC
   - AulasPUCMG: 60 commits, +4922 / -517 LOC
   - demo-ci: 56 commits, +692 / -274 LOC
   - DownloadGit: 2 commits, +342 / -7 LOC
   - extractgit: 1 commits, +271 / -0 LOC
   - GitBlame: 8 commits, +1774 / -789 LOC
   - JExpert: 1 commits, +1266 / -0 LOC
   - JExpert2.0: 1 commits, +991 / -0 LOC
   - jreuse: 12 commits, +2686 / -1397 LOC
   - JSERD2020: 43 commits, +1141 / -116 LOC
   - LineCounter: 2 commits, +217 / -0 LOC
   - ufmg.reuso.software: 58 commits, +32801 / -13513 LOC

🗂️ Dockerfile:
   - F03_API: 62 commits, 

In [14]:
output_dir = "graficosLoc"
os.makedirs(output_dir, exist_ok=True)

print("\n📊 Gerando gráficos de LOC por linguagem...")

for lang, repos in top5ReposLanguage.items():
    nomes_repos = []
    loc_totais = []

    for repo in repos:
        nomes_repos.append(repo['name'])
        total_loc = repo['locAdd'] + repo['locRemov']
        loc_totais.append(total_loc)

    plt.figure(figsize=(10, 6))

    bar_width = 0.4
    x_pos = range(len(nomes_repos))

    bars = plt.bar(x_pos, loc_totais, color='lightcoral', width=bar_width)

    plt.xticks(x_pos, nomes_repos, rotation=45, ha='right')
    plt.xlim(-0.5, len(nomes_repos) - 0.5)

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2., height,
                 f'{int(height)}',
                 ha='center', va='bottom')

    plt.title(f"LOC por repositório - {lang}")
    plt.xlabel("Repositórios")
    plt.ylabel("Linhas de Código (LOC)")
    plt.tight_layout()

    filepath = os.path.join(output_dir, f"{lang}_loc.png")
    plt.savefig(filepath)
    plt.close()

    print(f"✅ Gráfico salvo: {filepath}")



📊 Gerando gráficos de LOC por linguagem...
✅ Gráfico salvo: graficosLoc/Java_loc.png
✅ Gráfico salvo: graficosLoc/Dockerfile_loc.png
✅ Gráfico salvo: graficosLoc/Python_loc.png
✅ Gráfico salvo: graficosLoc/Makefile_loc.png
✅ Gráfico salvo: graficosLoc/AspectJ_loc.png


In [18]:
print(languageCommitCounter) #Lingugem e numero de commits
print(top5ReposLanguage) #Top5 linguagens com mais commits e repositórios respectivos com linhas add e remov
print(fotoPerfil)
print("/-" * 40)
print("\n📦 Estrutura detalhada de top5ReposLanguage:\n")

for lang, repos in top5ReposLanguage.items():
    print(f"🗂️ Linguagem: {lang}")
    for repo in repos:
        print(f"   🔸 Repositório: {repo['name']}")
        print(f"      ➕ Linhas adicionadas (locAdd): {repo['locAdd']}")
        print(f"      ➖ Linhas removidas (locRemov): {repo['locRemov']}")
    print("-" * 40)


Counter({'Java': 246, 'Dockerfile': 111, 'Python': 81, 'Makefile': 59, 'AspectJ': 58, 'Shell': 52, 'Jupyter Notebook': 7, 'C': 6, 'JavaScript': 4, 'HTML': 4, 'CSS': 4})
defaultdict(<class 'list'>, {'Java': [{'name': 'astar0', 'commits': 2, 'locAdd': 1649, 'locRemov': 31}, {'name': 'AulasPUCMG', 'commits': 60, 'locAdd': 4922, 'locRemov': 517}, {'name': 'demo-ci', 'commits': 56, 'locAdd': 692, 'locRemov': 274}, {'name': 'DownloadGit', 'commits': 2, 'locAdd': 342, 'locRemov': 7}, {'name': 'extractgit', 'commits': 1, 'locAdd': 271, 'locRemov': 0}, {'name': 'GitBlame', 'commits': 8, 'locAdd': 1774, 'locRemov': 789}, {'name': 'JExpert', 'commits': 1, 'locAdd': 1266, 'locRemov': 0}, {'name': 'JExpert2.0', 'commits': 1, 'locAdd': 991, 'locRemov': 0}, {'name': 'jreuse', 'commits': 12, 'locAdd': 2686, 'locRemov': 1397}, {'name': 'JSERD2020', 'commits': 43, 'locAdd': 1141, 'locRemov': 116}, {'name': 'LineCounter', 'commits': 2, 'locAdd': 217, 'locRemov': 0}, {'name': 'ufmg.reuso.software', 'commi